## Import Libraries and Clone Github

In [1]:
# Import Libraries
import os
from google.colab import userdata
import pandas as pd

In [2]:
# Import github token with google secrets thingy and clone git repository
GITHUB_TOKEN = userdata.get('github')
os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN
!git clone https://{GITHUB_TOKEN}@github.com/Kussil/Financial_Sentiment_LLM.git

fatal: destination path 'Financial_Sentiment_LLM' already exists and is not an empty directory.


In [3]:
# Define custom date parsing function as dates are in different formats sometimes
def parse_dates(date):
    """
    Parse dates from multiple formats into datetime objects.

    This function attempts to parse a date string into a pandas datetime object
    using a list of predefined formats. If the date string does not match any
    of the specified formats, it returns `pd.NaT`.

    Supported formats:
    - '%d-%b-%y' (e.g., '17-Aug-22')
    - '%m/%d/%Y' (e.g., '5/11/2024')
    - '%b-%d-%Y' (e.g., 'Apr-26-2024')

    Parameters:
    date (str): The date string to be parsed.

    Returns:
    pd.Timestamp or pd.NaT: The parsed datetime object if successful, or `pd.NaT` if parsing fails.
    """
    for fmt in ('%d-%b-%y', '%m/%d/%Y', '%b-%d-%Y'):
        try:
            return pd.to_datetime(date, format=fmt)
        except ValueError:
            pass
    return pd.NaT

## Upload Investment Research Articles into DF

In [4]:
# Import csvs
invest_df1 = pd.read_csv('Financial_Sentiment_LLM/01_Raw_Data/Investment_Research_ALLV2_Final_Trimmed_1.csv')
invest_df2 = pd.read_csv('Financial_Sentiment_LLM/01_Raw_Data/Investment_Research_ALLV2_Final_Trimmed_2.csv')
invest_df = pd.concat([invest_df1, invest_df2], axis=0, ignore_index=True)

# Rename drop, and reorder columns
invest_df = invest_df.rename(columns={'Headline': 'Article Headline', 'Text': 'Article Text'})
invest_df = invest_df.drop(['Contributor', 'Date/Time'], axis=1)
invest_df['Source'] = 'Investment Research'
new_order = ['Source', 'Unique_ID', 'Ticker', 'Date', 'Article Headline', 'Article Text', 'URL']
invest_df = invest_df.reindex(columns=new_order)

# Remove periods and trim whitespace
invest_df['Ticker'] = invest_df['Ticker'].str.replace('.', '', regex=False).str.strip()

# Add unique ID since it is missing
invest_df['Unique_ID'] = ['IR-' + str(i) for i in range(1, len(invest_df) + 1)]

# Fix Date format
invest_df['Date'] = pd.to_datetime(invest_df['Date'])

display(invest_df.shape)
display(invest_df.isna().sum())
display(invest_df.head())

(4896, 7)

Source                 0
Unique_ID              0
Ticker                 0
Date                   0
Article Headline       0
Article Text          16
URL                 4896
dtype: int64

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
0,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN
1,Investment Research,IR-2,EOG,2024-05-14,"EOG Resources, Inc.","Stock Report | May 14, 2024 | NYSESymbol: EOG ...",NaN
2,Investment Research,IR-3,EOG,2024-05-11,"EOG Resources, Inc.","Stock Report | May 11, 2024 | NYSESymbol: EOG ...",NaN
3,Investment Research,IR-4,DVN,2024-05-11,Devon Energy Corporation,"Stock Report | May 11, 2024 | NYSESymbol: DVN ...",NaN
4,Investment Research,IR-5,COP,2024-05-07,ConocoPhillips,"Stock Report | May 07, 2024 | NYSESymbol: COP ...",NaN


In [5]:
# Drop rows with no article text
invest_df = invest_df.dropna(subset=['Article Text'])
display(invest_df.shape)

(4880, 7)

## Upload Proquest News Articles into DF and Clean Data

In [6]:
# Import Files
proquest_df_cvx = pd.read_csv('Financial_Sentiment_LLM/01_Raw_Data/Proquest_Articles_CVX.csv')
proquest_df_xom = pd.read_csv('Financial_Sentiment_LLM/01_Raw_Data/ProQuest_Articles_XOM.csv')
proquest_df_others = pd.read_csv('Financial_Sentiment_LLM/01_Raw_Data/ProQuest_Articles_FINAL_BP_COP_OXY_SHEL.csv')
proquest_df_others2 = pd.read_csv('Financial_Sentiment_LLM/01_Raw_Data/ProQuest_Articles_FINAL_CXO_DVN_EOG_EQNR_HES_MPC_MRO_PSX_PXD_TTL.csv')

# Append dfs
proquest_df = pd.concat([proquest_df_cvx,
                         proquest_df_xom,
                         proquest_df_others,
                         proquest_df_others2
                         ], ignore_index=True)
display(proquest_df_cvx.shape)
display(proquest_df_xom.shape)
display(proquest_df_others.shape)
display(proquest_df_others2.shape)
display(proquest_df.shape)

# Rename drop, and reorder columns
proquest_df = proquest_df.rename(columns={'Title': 'Article Headline', 'Full Article Text': 'Article Text'})
proquest_df['Source'] = 'ProQuest'
proquest_df = proquest_df.reindex(columns=new_order)

# Replace TTL with TTE
proquest_df['Ticker'] = proquest_df['Ticker'].str.replace('TTL', 'TTE')

# Remove periods and trim whitespace
proquest_df['Ticker'] = proquest_df['Ticker'].str.replace('.', '', regex=False).str.strip()

# Prepend the Unique ID to ensure no duplicates
proquest_df['Unique_ID'] = 'PQ-' + proquest_df['Unique_ID'].astype(str)

# Fix Date format
proquest_df['Date'] = proquest_df['Date'].apply(parse_dates)

display(proquest_df.isna().sum())
display(proquest_df.head())

(261, 6)

(1478, 6)

(1445, 6)

(932, 6)

(4116, 6)

Source                 0
Unique_ID              0
Ticker                 0
Date                2377
Article Headline       0
Article Text           0
URL                    0
dtype: int64

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
0,ProQuest,PQ-2703147472,CVX,2022-08-17,Oil Giants Must Face Climate-Liability Suits i...,Turn on search term navigationTurn on search t...,https://www.proquest.com/newspapers/chevron-ge...
1,ProQuest,PQ-2702197510,CVX,2022-08-15,Warren Buffett's Berkshire Hathaway Keeps Spen...,Turn on search term navigationTurn on search t...,https://www.proquest.com/newspapers/chevron-sl...
2,ProQuest,PQ-2697049245,CVX,2022-08-02,"Investors Put Forward More Proposals, Dialing ...",Turn on search term navigationTurn on search t...,https://www.proquest.com/newspapers/chevron-sl...
3,ProQuest,PQ-2696371267,CVX,2022-07-30,Soaring Fuel Prices Catapult Oil Giants to Rec...,Turn on search term navigationTurn on search t...,https://www.proquest.com/newspapers/senators-a...
4,ProQuest,PQ-2695835797,CVX,2022-07-29,"Exxon, Chevron, Shell Report Record Profits on...",Turn on search term navigationTurn on search t...,https://www.proquest.com/newspapers/shell-trie...


In [7]:
# Delete rows with missing article text
search_text_1 = 'Failed to load content: Message:'
search_text_2 = 'Relevant content not found within the specified range.'
search_text_3 = 'TVM:UNDEFINED'

# Count the number of rows containing either search text
count_rows = proquest_df[
    proquest_df['Article Text'].str.contains(search_text_1, na=False) |
    proquest_df['Article Text'].str.contains(search_text_2, na=False) |
    proquest_df['Article Text'].str.contains(search_text_3, na=False)
].shape[0]

# Delete the rows containing either search text
proquest_df = proquest_df[
    ~proquest_df['Article Text'].str.contains(search_text_1, na=False) &
    ~proquest_df['Article Text'].str.contains(search_text_2, na=False) &
    ~proquest_df['Article Text'].str.contains(search_text_3, na=False)
]

# Print the number of rows with missing article text and the new shape of the DataFrame
print(f"Number of rows with missing article text: {count_rows}")
print(f"New DataFrame shape: {proquest_df.shape}")
print()

# Confirm data is good by looking for short article headlines
shortest_headline = proquest_df.loc[proquest_df['Article Headline'].str.len().idxmin(), 'Article Headline']
print(f"The shortest article headline is: '{shortest_headline}'")
print()

# Confirm data is good by looking for short article text
shortest_text = proquest_df.loc[proquest_df['Article Text'].str.len().idxmin(), 'Article Text']
print(f"The shortest article text is: '{shortest_text}'")

Number of rows with missing article text: 233
New DataFrame shape: (3883, 7)

The shortest article headline is: 'BP PLC'

The shortest article text is: 'Turn on search term navigationTurn on search term navigation
| Jump to first hitOKLAHOMA CITY Devon Energy Corp.’s board of directors has declared a quarterly cash dividend of 9 cents per share on common stock for the first quarter of 2020. The dividend is payable on March 31 based on a record date of March 13.
'


In [8]:
# Clean extra text from the beginning of the article
def clean_text(text):
    """
    Cleans a given text by removing a specific leading substring and stripping whitespace.

    This function checks if the provided text starts with the substring "Turn on search term". If it does, the function
    removes the first 80 characters from the text and then strips any leading or trailing whitespace from the remaining text.
    If the text does not start with the specified substring, it returns the text unchanged.

    Parameters:
    text (str): The text to be cleaned.

    Returns:
    str: The cleaned text.
    """
    if text.startswith("Turn on search term"):
            return text[80:].strip()
    return text
proquest_df['Article Text'] = proquest_df['Article Text'].apply(clean_text)

# Confirm data is good by looking for short article headlines
shortest_headline = proquest_df.loc[proquest_df['Article Headline'].str.len().idxmin(), 'Article Headline']
print(f"The shortest article headline is: '{shortest_headline}'")
print()

# Confirm data is good by looking for short article text
shortest_text = proquest_df.loc[proquest_df['Article Text'].str.len().idxmin(), 'Article Text']
print(f"The shortest article text is: '{shortest_text}'")

display(proquest_df.head())

The shortest article headline is: 'BP PLC'

The shortest article text is: 'OKLAHOMA CITY Devon Energy Corp.’s board of directors has declared a quarterly cash dividend of 9 cents per share on common stock for the first quarter of 2020. The dividend is payable on March 31 based on a record date of March 13.'


,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
0,ProQuest,PQ-2703147472,CVX,2022-08-17,Oil Giants Must Face Climate-Liability Suits i...,Chevron Corp.\nSpending projections are slidin...,https://www.proquest.com/newspapers/chevron-ge...
1,ProQuest,PQ-2702197510,CVX,2022-08-15,Warren Buffett's Berkshire Hathaway Keeps Spen...,Chevron Corp. said it would cut its annual cap...,https://www.proquest.com/newspapers/chevron-sl...
2,ProQuest,PQ-2697049245,CVX,2022-08-02,"Investors Put Forward More Proposals, Dialing ...",Chevron Corp. said it would cut its annual cap...,https://www.proquest.com/newspapers/chevron-sl...
3,ProQuest,PQ-2696371267,CVX,2022-07-30,Soaring Fuel Prices Catapult Oil Giants to Rec...,SENATORS aired misgivings on Tuesday over pote...,https://www.proquest.com/newspapers/senators-a...
4,ProQuest,PQ-2695835797,CVX,2022-07-29,"Exxon, Chevron, Shell Report Record Profits on...",LONDON—Royal Dutch Shell PLC said it would sta...,https://www.proquest.com/newspapers/shell-trie...


In [9]:
# Look for duplicate rows
duplicates = proquest_df[proquest_df.duplicated(subset=['Ticker', 'Date', 'Article Headline'], keep=False)]
duplicate_count = duplicates.shape[0]
print(f"Number of duplicate rows: {duplicate_count}")
display(duplicates.head(6))
print()

# Drop duplicates and keep the first occurrence
proquest_df = proquest_df.drop_duplicates(subset=['Ticker', 'Date', 'Article Headline'], keep='first')
display(proquest_df.shape)

Number of duplicate rows: 390


,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
69,ProQuest,PQ-2774491352,CVX,2021-05-27,Chevron corp. investors back climate proposal;...,Russia is still finding a home for much of its...,https://www.proquest.com/newspapers/business-n...
70,ProQuest,PQ-2533063981,CVX,2021-05-27,Chevron corp. investors back climate proposal;...,Russia is still finding a home for much of its...,https://www.proquest.com/newspapers/chevron-ce...
94,ProQuest,PQ-2483968356,CVX,2021-01-30,Chevron posts fourth-quarter loss on weak refi...,[Financial Analysis and Commentary]\nChevron w...,https://www.proquest.com/newspapers/exchange-h...
95,ProQuest,PQ-2483951085,CVX,2021-01-30,Chevron posts fourth-quarter loss on weak refi...,Chevron Corp. had its most profitable year sin...,https://www.proquest.com/newspapers/exchange-c...
105,ProQuest,PQ-2455124806,CVX,2020-10-28,Chevron to lay off about 25% of noble energy e...,Michael Wirth Chevron Corporation has issued a...,https://www.proquest.com/newspapers/chevron-se...
106,ProQuest,PQ-2455119828,CVX,2020-10-28,Chevron to lay off about 25% of noble energy e...,"Chevron Corp. committed to an ""aspiration"" of ...",https://www.proquest.com/newspapers/chevron-em...


(3635, 7)

## Upload SEC Filings into DF

In [10]:
# Import Chevron and Other Majors CSVs
sec_df = pd.read_csv('Financial_Sentiment_LLM/01_Raw_Data/SEC_filings.csv')

# # Rename drop, and reorder columns
sec_df = sec_df.rename(columns={'AccessionNumber': 'Unique_ID', 'Form': 'Article Headline', 'Text': 'Article Text'})
sec_df['Source'] = 'SEC Filings'
sec_df = sec_df.reindex(columns=new_order)

# Remove periods and trim whitespace
sec_df['Ticker'] = sec_df['Ticker'].str.replace('.', '', regex=False).str.strip()

# Prepend the Unique ID to ensure no duplicates
sec_df['Unique_ID'] = 'SEC-' + sec_df['Unique_ID'].astype(str)

# Fix Date format
sec_df['Date'] = pd.to_datetime(sec_df['Date'])

display(sec_df.shape)
display(sec_df.isna().sum())
display(sec_df.head())

(1031, 7)

Source              0
Unique_ID           0
Ticker              0
Date                0
Article Headline    0
Article Text        0
URL                 0
dtype: int64

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
0,SEC Filings,SEC-114036119010687,BP,2019-06-07,11-K,\nSECURITIES AND EXCHANGE COMMISSION\nWashingt...,https://www.sec.gov/Archives/edgar/data/000031...
1,SEC Filings,SEC-31380720000008,BP,2020-06-23,11-K,UNITED STATESSECURITIES AND EXCHANGE COMMISSIO...,https://www.sec.gov/Archives/edgar/data/000031...
2,SEC Filings,SEC-31380721000009,BP,2021-06-11,11-K,UNITED STATESSECURITIES AND EXCHANGE COMMISSIO...,https://www.sec.gov/Archives/edgar/data/000031...
3,SEC Filings,SEC-31380722000025,BP,2022-06-10,11-K,UNITED STATESSECURITIES AND EXCHANGE COMMISSIO...,https://www.sec.gov/Archives/edgar/data/000031...
4,SEC Filings,SEC-31380723000020,BP,2023-06-13,11-K,UNITED STATESSECURITIES AND EXCHANGE COMMISSIO...,https://www.sec.gov/Archives/edgar/data/000031...


## Upload Earnings Call Transcripts into DF

In [11]:
# Import csvs
earnings1 = pd.read_csv('/content/Financial_Sentiment_LLM/01_Raw_Data/Earnings Transcripts/Chevron Corporation NYSE CVX Transcripts_ALL_Final.csv')
earnings2 = pd.read_csv('/content/Financial_Sentiment_LLM/01_Raw_Data/Earnings Transcripts/EarningsTranscripts_DVN_HES_MPC_MRO_OXY_VLO_PSX_PDC_CXO_ALL_Final.csv')
earnings3 = pd.read_csv('/content/Financial_Sentiment_LLM/01_Raw_Data/Earnings Transcripts/EarningsTranscripts_XOM_TTE_COP_EOG_BP_SHEL_ALL_Final.csv')
earnings_presentations = pd.concat([earnings1, earnings2, earnings3], axis=0, ignore_index=True)

# Split into presentation and Q&A
earnings_qa = earnings_presentations.copy()
earnings_presentations = earnings_presentations.drop(['Question and Answer Text'], axis=1)
earnings_qa = earnings_qa.drop(['Presentation Text'], axis=1)

# Remove periods and trim whitespace
earnings_presentations['Ticker'] = earnings_presentations['Ticker'].str.replace('.', '', regex=False).str.strip()
earnings_qa['Ticker'] = earnings_qa['Ticker'].str.replace('.', '', regex=False).str.strip()

# Fix Date format
earnings_presentations['Date'] = earnings_presentations['Date'].apply(parse_dates)
earnings_qa['Date'] = earnings_qa['Date'].apply(parse_dates)

display(earnings_presentations.head(1))
display(earnings_qa.head(1))

,Date / Time,Title,Type,Date,Company,Ticker,Presentation Text
0,Apr-26-2024 11:00 AM,"Chevron Corporation, Q1 2024 Earnings Call, Ap...",Earnings Call,2024-04-26,Chevron Corporation,CVX,Chevron Corporation NYSE:CVX\nFQ1 2024 Earning...


,Date / Time,Title,Type,Date,Company,Ticker,Question and Answer Text
0,Apr-26-2024 11:00 AM,"Chevron Corporation, Q1 2024 Earnings Call, Ap...",Earnings Call,2024-04-26,Chevron Corporation,CVX,Question and Answer\nOperator\n[Operator Instr...


In [12]:
# Rename, drop, and reorder columns.  Also add a unique ID
earnings_presentations = earnings_presentations.rename(columns={'Title': 'Article Headline', 'Presentation Text': 'Article Text'})
earnings_qa = earnings_qa.rename(columns={'Title': 'Article Headline', 'Question and Answer Text': 'Article Text'})

earnings_presentations = earnings_presentations.drop(['Type', 'Date / Time'], axis=1)
earnings_qa = earnings_qa.drop(['Type', 'Date / Time'], axis=1)

earnings_presentations['Source'] = 'Earnings Call Presentations'
earnings_qa['Source'] = 'Earnings Call Q&A'

earnings_presentations = earnings_presentations.reindex(columns=new_order)
earnings_qa = earnings_qa.reindex(columns=new_order)

earnings_presentations['Unique_ID'] = ['EP-' + str(i) for i in range(1, len(earnings_presentations) + 1)]
earnings_qa['Unique_ID'] = ['EQ-' + str(i) for i in range(1, len(earnings_qa) + 1)]

display(earnings_presentations.shape)
display(earnings_presentations.isna().sum())
display(earnings_presentations.head())

display(earnings_qa.shape)
display(earnings_qa.isna().sum())
display(earnings_qa.head())

(342, 7)

Source                0
Unique_ID             0
Ticker               25
Date                  0
Article Headline      0
Article Text         73
URL                 342
dtype: int64

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
0,Earnings Call Presentations,EP-1,CVX,2024-04-26,"Chevron Corporation, Q1 2024 Earnings Call, Ap...",Chevron Corporation NYSE:CVX\nFQ1 2024 Earning...,NaN
1,Earnings Call Presentations,EP-2,CVX,2024-02-02,"Chevron Corporation, Q4 2023 Earnings Call, Fe...",Chevron Corporation NYSE:CVX\nFQ4 2023 Earning...,NaN
2,Earnings Call Presentations,EP-3,CVX,2023-10-27,"Chevron Corporation, Q3 2023 Earnings Call, Oc...",Chevron Corporation NYSE:CVX\nFQ3 2023 Earning...,NaN
3,Earnings Call Presentations,EP-4,CVX,2023-07-28,"Chevron Corporation, Q2 2023 Earnings Call, Ju...",Chevron Corporation NYSE:CVX\nFQ2 2023 Earning...,NaN
4,Earnings Call Presentations,EP-5,CVX,2023-04-28,"Chevron Corporation, Q1 2023 Earnings Call, Ap...",Chevron Corporation NYSE:CVX\nFQ1 2023 Earning...,NaN


(342, 7)

Source                0
Unique_ID             0
Ticker               25
Date                  0
Article Headline      0
Article Text         80
URL                 342
dtype: int64

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
0,Earnings Call Q&A,EQ-1,CVX,2024-04-26,"Chevron Corporation, Q1 2024 Earnings Call, Ap...",Question and Answer\nOperator\n[Operator Instr...,NaN
1,Earnings Call Q&A,EQ-2,CVX,2024-02-02,"Chevron Corporation, Q4 2023 Earnings Call, Fe...",Question and Answer\nOperator\n[Operator Instr...,NaN
2,Earnings Call Q&A,EQ-3,CVX,2023-10-27,"Chevron Corporation, Q3 2023 Earnings Call, Oc...",Question and Answer\nOperator\n[Operator Instr...,NaN
3,Earnings Call Q&A,EQ-4,CVX,2023-07-28,"Chevron Corporation, Q2 2023 Earnings Call, Ju...",Question and Answer\nOperator\n[Operator Instr...,NaN
4,Earnings Call Q&A,EQ-5,CVX,2023-04-28,"Chevron Corporation, Q1 2023 Earnings Call, Ap...",Question and Answer\nOperator\n[Operator Instr...,NaN


In [13]:
# Drop rows with no article text or ticker
earnings_presentations = earnings_presentations.dropna(subset=['Article Text', 'Ticker'])
display(earnings_presentations.shape)

earnings_qa = earnings_qa.dropna(subset=['Article Text', 'Ticker'])
display(earnings_qa.shape)

(257, 7)

(250, 7)

## Concatenate Sources and Export to CSV

In [14]:
# Concatenate
text_df = pd.concat([invest_df, proquest_df, sec_df, earnings_presentations, earnings_qa], ignore_index=True)
display(text_df.shape)
display(text_df.head())
display(text_df.tail())

(10053, 7)

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
0,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN
1,Investment Research,IR-2,EOG,2024-05-14,"EOG Resources, Inc.","Stock Report | May 14, 2024 | NYSESymbol: EOG ...",NaN
2,Investment Research,IR-3,EOG,2024-05-11,"EOG Resources, Inc.","Stock Report | May 11, 2024 | NYSESymbol: EOG ...",NaN
3,Investment Research,IR-4,DVN,2024-05-11,Devon Energy Corporation,"Stock Report | May 11, 2024 | NYSESymbol: DVN ...",NaN
4,Investment Research,IR-5,COP,2024-05-07,ConocoPhillips,"Stock Report | May 07, 2024 | NYSESymbol: COP ...",NaN


,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
10048,Earnings Call Q&A,EQ-338,XOM,2021-02-02,"Exxon Mobil Corporation, Q4 2020 Earnings Call...",Question and Answer\nOperator\n[Operator Instr...,NaN
10049,Earnings Call Q&A,EQ-339,COP,2021-02-02,"ConocoPhillips, Q4 2020 Earnings Call, Feb 02,...",Question and Answer\nOperator\n[Operator Instr...,NaN
10050,Earnings Call Q&A,EQ-340,EOG,2019-05-03,"EOG Resources, Inc., Q1 2019 Earnings Call, Ma...",Question and Answer\nOperator\n[Operator Instr...,NaN
10051,Earnings Call Q&A,EQ-341,SHEL,2019-05-02,"Royal Dutch Shell plc, Q1 2019 Earnings Call, ...",Question and Answer\nOperator\n[Operator Instr...,NaN
10052,Earnings Call Q&A,EQ-342,COP,2019-04-30,"ConocoPhillips, Q1 2019 Earnings Call, Apr 30,...",Question and Answer\nOperator\n[Operator Instr...,NaN


In [15]:
# Get a count of articles by source and ticker
grouped_counts = text_df.groupby(['Source', 'Ticker']).size().reset_index(name='Count')
pivot_df = grouped_counts.pivot(index='Ticker', columns='Source', values='Count')
pivot_df = pivot_df.fillna(0)
display(pivot_df)

Source,Earnings Call Presentations,Earnings Call Q&A,Investment Research,ProQuest,SEC Filings
Ticker,,,,,
BP,16.0,16.0,276.0,488.0,5.0
COP,16.0,16.0,282.0,328.0,80.0
CVX,22.0,22.0,362.0,220.0,100.0
CXO,0.0,0.0,143.0,28.0,0.0
DVN,19.0,19.0,279.0,107.0,75.0
EOG,16.0,16.0,294.0,6.0,87.0
EQNR,0.0,0.0,4.0,407.0,0.0
HES,19.0,19.0,279.0,18.0,77.0
MPC,19.0,19.0,346.0,24.0,104.0


In [16]:
# Export as CSV
# Note the consolidated file is too big now, will need to be consolidated in a df when used
# text_df.to_csv('/content/Consolidated_Text_Data.csv', index=False)

# Split the investment research df due to size
midpoint = len(invest_df) // 2
invest_df1 = invest_df.iloc[:midpoint]
invest_df2 = invest_df.iloc[midpoint:]

# Exporting separate files
invest_df1.to_csv('/content/Investment_Research_Part1.csv', index=False)
invest_df2.to_csv('/content/Investment_Research_Part2.csv', index=False)
proquest_df.to_csv('/content/ProQuest_Articles.csv', index=False)
sec_df.to_csv('/content/SEC_Filings.csv', index=False)
earnings_presentations.to_csv('/content/Earnings_Presentations.csv', index=False)
earnings_qa.to_csv('/content/Earnings_QA.csv', index=False)

# Note: These exports needs to be manually uploaded to Github.